In [3]:
import cupy as cp
# import numpy as np
import pandas as pd

def normalize(arr):
    for x in arr:
        if x.max() - x.min() == 0:
            normalized = (x - x.min()) / x.max()
        else:
            normalized = (x - x.min()) / (x.max() - x.min())
        x[:] = normalized[:]
df = pd.read_csv('/content/drive/MyDrive/Диплом/data/total_df_for_aio_chickpea_28042016_synchro.csv', usecols=range(1, 10))#6644))#, nrows=200)
feature_names_list = []
for x in df.columns:
    feature_names_list.append(x)
    # print(f"'{x}'", end = ", ")
arr = df.values

filename = '/content/drive/MyDrive/Диплом/data/memmapped.dat'

# mmapped_array = numpy.memmap(filename, dtype=arr.dtype, mode='w+', shape=arr.shape)
# mmapped_array[:] = arr[:]

X = arr

X = X.T
normalize(X)

# print(mmapped_array.shape)
# print(mmapped_array.dtype)
# for x in mmapped_array:
#     print(*x)









ModuleNotFoundError: No module named 'cupy'

In [12]:

def two_d_norm(xy):
    # xy is N x 2 xy cordinates, returns normed-xy on [0,1]
    norm_xy = (xy - xy.min(axis = 0)) / (xy - xy.min(axis = 0)).max(axis = 0)
    return norm_xy

def two_d_eq(xy,Nn):
    # xy is N x 2 xy cordinates, returns eq-xy on [0,1]
    xx_rank = cp.argsort(xy[:,0])
    yy_rank = cp.argsort(xy[:,1])
    eq_xy = cp.full(xy.shape,cp.nan)
    for ii in range(xy.shape[0]):
        xx_idx = xx_rank[ii]
        yy_idx = yy_rank[ii]
        eq_xy[xx_idx,0] = ii * 1/Nn
        eq_xy[yy_idx,1] = ii * 1/Nn
    return eq_xy

#embedding = MDS(n_components=2)
#mds_xy = embedding.fit_transform(transposed_input)
# to pixels
def Assign_features_to_pixels(xy,nn,verbose = False):
    # For each unassigned feature, find its nearest pixel, repeat until every ft is assigned
    # xy is the 2-d coordinates (normalized to [0,1]); nn is the image width. Img size = n x n
    # generate the result summary table, xy pixels; 3rd is nan for filling the idx
    Nn = xy.shape[0]

    from itertools import product
    pixel_iter = product([x for x in range(nn)],repeat = 2)
    result_table = cp.full((nn*nn,3),cp.nan)
    ii = 0
    for each_pixel in pixel_iter:
        result_table[ii,:2] = cp.array(each_pixel)
        ii+=1
    # Use numpy array for speed



#    xy = eq_xy
    centroids = result_table[:,:2] / nn + 0.5/nn
    pixel_avail = cp.ones(nn*nn).astype(bool)
    feature_assigned = cp.zeros(Nn).astype(bool)

    # from sklearn.metrics import pairwise_distances

    def pairwise_distances_cupy(centroids, xy):
        dists = cp.sqrt(cp.sum((centroids[:, cp.newaxis] - xy) ** 2, axis=2))
        return dists

    dist_xy_centroids = pairwise_distances_cupy(centroids,xy)

    while feature_assigned.sum()<Nn:
        # Init the pick-relationship table
        pick_xy_centroids = cp.zeros(dist_xy_centroids.shape).astype(bool)

        for each_feature in range(Nn):
            # for each feature, find the nearest available pixel
            if feature_assigned[each_feature] == True:
                # if this feature is already assigned, skip to the next ft
                continue
            else:
                # if not assigned:
                for ii in range(nn*nn):
                    # find the nearest avail pixel
                    nearest_pixel_idx = cp.argsort(dist_xy_centroids[:,each_feature])[ii]
                    if pixel_avail[nearest_pixel_idx] == True:
                        break
                    else:
                        continue
                pick_xy_centroids[nearest_pixel_idx,each_feature] = True

        for each_pixel in range(nn*nn):
            # Assign the feature No to pixels
            if pixel_avail[each_pixel] == False:
                continue
            else:
                # find all the "True" features. np.where returns a tuple size 1
                related_features = cp.where(pick_xy_centroids[each_pixel,:] == 1)[0]
                if len(related_features) == 1:
                    # Assign it
                    result_table[each_pixel,2] = related_features[0]
                    pixel_avail[each_pixel] = False
                    feature_assigned[related_features[0]] = True
                elif len(related_features) > 1:
                    related_dists = dist_xy_centroids[each_pixel,related_features]
                    best_feature = related_features[cp.argsort(related_dists)[0]] # Sort, and pick the nearest one among them
                    result_table[each_pixel,2] = best_feature
                    pixel_avail[each_pixel] = False
                    feature_assigned[best_feature] = True
        if verbose:
            print(">> Assign features to pixels:", feature_assigned.sum(),"/",Nn)
    result_table = result_table.astype(int)

    img = cp.full((nn,nn),cp.nan)#.astype(object)
    for each_pixel in range(nn*nn):
        xx = result_table[each_pixel,0]
        yy = result_table[each_pixel,1]
        # ft = 'F' + str(result_table[each_pixel,2])
        ft = result_table[each_pixel,2]
        img[xx,yy] = ft
    return img#.astype(object)
# print(">>>> MDS")
#eq_xy = two_d_eq(mds_xy)
#Img = Assign_features_to_pixels(eq_xy,nn,verbose=1)
#Init_Corr_MDS = InitCorr(dist_mat,Img,nn)

def MDS_Im_Gen(X,nn, Img):
    [N_sam,P_Feat] = X.shape
    X_Gen = cp.zeros((N_sam,nn**2))
    conv_Img = Img.reshape(Img.size,1)
    for i in range(nn**2):
        Feature = cp.array(conv_Img[i]);    Feature = Feature[0]; F_Num = int(Feature[1:])
        if abs(F_Num) < nn**2:
            X_Gen[:,i] = X[:,F_Num]
        else:
            X_Gen[:,i] = 0
    return X_Gen

#%% CCLE functions
def dataframer(Main,Set_in, name_in, name_out):
    A = Set_in[name_in].tolist()
    Set_out = Main[Main[name_out] == A[0]]
    for cell in range(len(A) - 1):
        df = Main[Main[name_out] == A[cell + 1]]
        Set_out = pd.concat([Set_out, df])
    return Set_out

def REFINED_Im_Gen(X,nn, map_in_int, gene_names,coords):
	[N_sam,P_Feat] = X.shape
	X_Gen = cp.zeros((N_sam,nn**2))
	for i in range(N_sam):
		data = X[i,:]
		X_REFINED = pd.DataFrame(data = data.reshape(1,len(data)), columns = gene_names)
		Image = cp.zeros(map_in_int.shape)
		for j in range(len(coords)):
			val = cp.array(X_REFINED[gene_names[j]])
			Image[coords[j,0],coords[j,1]] = val
		Image = Image.reshape(nn**2)
		X_Gen[i,:] = Image
	return X_Gen


def myMDS(dist, m = 2):
    n = dist.shape[0]
    c = cp.eye(n) - cp.ones((n, n)) / n
    b = cp.dot(c, cp.dot(dist, c)) * (-0.5)
    eigenvalues, eigenvectors = cp.linalg.eigh(b)
    indexes = cp.argsort(eigenvalues)[-m:]
    eigenvalues = cp.array([eigenvalues[index] for index in indexes])
    eigenvectors = cp.array([eigenvectors[index] for index in indexes])
    eigenvalues = cp.real(eigenvalues)
    eigenvectors = cp.real(eigenvectors)
    return cp.dot(eigenvectors.T, cp.diag(eigenvalues ** 0.5))


In [28]:
from scipy.stats import pearsonr
# import numpy as np
import random
from scipy.spatial import distance
import pickle
import pandas as pd
import time
from itertools import product

#%%
def pairwise_distances(points):
        num_points = points.shape[0]
        distances = cp.zeros((num_points, num_points))

        for i in range(num_points):
            for j in range(i+1, num_points):
                distance = cp.linalg.norm(points[i] - points[j])
                distances[i, j] = distance
                distances[j, i] = distance

        return distances

def custom_squareform(dist_matrix):
    n = dist_matrix.shape[0]
    upper_indices = cp.triu_indices(n, k=1)
    squareform = dist_matrix[upper_indices]
    return squareform



def universial_corr(dist_matr, mapping_in_int):
    # print(dist_matr)

    print(mapping_in_int)
    # dist_matr is a sqr matr
    Nn = dist_matr.shape[0]
    # find what is the int coordinates for each feature, get an array
    # Because np.where returns a tuple (x_position array, y_position array), a generation is used
    # print(Nn)
    # coord = cp.ndarray((2, Nn))
    coord = []
    for i in range(Nn):
        print(i)
        print(cp.where(mapping_in_int == i))

        coord.append(cp.where(mapping_in_int == i))
    print()
    # print(*coord, end="\n")
    coord = cp.asarray(coord)
    # print(type([[item for item in cp.where(mapping_in_int == ii)] for ii in range(Nn)][0][0]))
    # get a 1-d form distance the euclidean dist between pixles positions
    pixel_dist = custom_squareform(cp.linalg.norm(coord[:, None] - coord, axis=2))
    # pixel_dist = pairwise_distances(coord)
    pixel_dist = pixel_dist.reshape(len(pixel_dist),1)
    # convert the 2-d distance to 1d distance
    feature_dist = custom_squareform(dist_matr)
    feature_dist = feature_dist.reshape(len(feature_dist),1)
    ## pearsonr returns a tuple
    #corrs = pearsonr(feature_dist,pixel_dist)[0]
    L2_Norm = cp.sqrt(sum((pixel_dist - feature_dist)**2)/sum(feature_dist**2))
    # print(L2_Norm)
    return L2_Norm
#%%
def evaluate_swap(coord1,coord2,dist_matr,mapping_in_int,original_corr = -2):
    # Coord are in list[]
    # Avoid messing up with the origianl map
    # The correlation before swap can be passed to save some calculation
    the_map = mapping_in_int.copy()
    # If out of bound, return NaN.
    if coord1[0]<0 or coord1[1]<0 or coord2[0]<0 or coord2[1]<0:
        return cp.nan
    if coord1[0]>=the_map.shape[0] or coord1[1]>=the_map.shape[0] or coord2[0]>=the_map.shape[0] or coord2[1]>=the_map.shape[0]:
        return cp.nan
    # If not given, recompute.
    if original_corr<-1 or original_corr>1:
        original_corr = universial_corr(dist_matr,the_map)
    # Swap
    try:
        temp = the_map[coord1[0],coord1[1]]
        the_map[coord1[0],coord1[1]] = the_map[coord2[0],coord2[1]]
        the_map[coord2[0],coord2[1]] = temp
        changed_corr = universial_corr(dist_matr,the_map)
        return(changed_corr - original_corr)[0]
    except IndexError:
        raise Warning ("Swap index:", coord1,coord2,"Index error. Check the coordnation.")


def evaluate_centroid(centroid,dist_matr,mapping_in_int):
    original_corr = universial_corr(dist_matr,mapping_in_int)
    results = [100000] # just to skip the 0 position
    for each_direc in product([-1,0,1],repeat = 2):
        #print(each_direc)
        # directions are returned as tuple (-1,1), (-1,0), (-1,1), (0,0), ....
        swap_coord = [centroid[0]+each_direc[0],centroid[1]+each_direc[1]]
        evaluation = evaluate_swap(centroid,swap_coord,dist_matr,mapping_in_int,original_corr)
        results.append(evaluation)
    results_array = cp.array(results)
    #best_swap_direc = np.where(results_array == np.nanmax(results_array))[0][0]
    best_swap_direc = cp.where(results_array == cp.nanmin(results_array))[0][0]
    # Give the best direction as a int
    return best_swap_direc

def evaluate_centroids_in_list(centroids_list,dist_matr,mapping_in_int):
    # and returns a dict
    results = dict()
    for each_centr in centroids_list:
        each_centr = tuple(each_centr)
        evaluation = evaluate_centroid(each_centr,dist_matr,mapping_in_int)
        results.update({each_centr:evaluation})
    return results

#%%
def execute_coordination_swap(coord1,coord2,mapping_in_int):
    # try passing the ref. directly
    the_map = mapping_in_int#.copy()
    # If out of bound, return NaN.
    if coord1[0]<0 or coord1[1]<0 or coord2[0]<0 or coord2[1]<0:
        raise Warning("Swapping failed:",coord1,coord2,"-- Negative coordnation.")
        return the_map
    if coord1[0]>the_map.shape[0] or coord1[1]>the_map.shape[0] or coord2[0]>the_map.shape[0] or coord2[1]>the_map.shape[0]:
        raise Warning("Swapping failed:",coord1,coord2,"-- Coordnation out of bound.")
        return the_map

    temp = the_map[coord1[0],coord1[1]]
    the_map[coord1[0],coord1[1]] = the_map[coord2[0],coord2[1]]
    the_map[coord2[0],coord2[1]] = temp

    return(the_map)

# Initial centriod id & Swapping direction:
# 1 2 3
# 4 5 6
# 7 8 9
# 0 in swapping is preserved for the header.

def execute_direction_swap(centroid,mapping_in_int,direction = 5):
    # Need to notice that [0] is the vertival coord, [1] is the horiz coord. similar to the matlab images.
    coord1 = list(centroid)
    coord2 = list(centroid)
    if direction not in range(1,10):
        print(direction)
        raise ValueError("Invalid swapping direction.")
    if direction == 5:
        return mapping_in_int

    if direction in [1,4,7]:
        coord2[1] -=1
    elif direction in [3,6,9]:
        coord2[1] +=1

    if direction in [1,2,3]:
        coord2[0] -=1
    elif direction in [7,8,9]:
        coord2[0] +=1

    the_map = execute_coordination_swap(coord1,coord2,mapping_in_int)
    return the_map

def execute_dict_swap(swapping_dict, mapping_in_int):
    for each_key in swapping_dict:
        execute_direction_swap(each_key,mapping_in_int,direction = swapping_dict[each_key])
    return mapping_in_int

In [6]:
iters_count = 10
path = "/content/drive/MyDrive/Диплом/data/res"
import time
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
# from Toolbox import two_d_eq, Assign_features_to_pixels
from sklearn.metrics.pairwise import euclidean_distances
import math
from itertools import product


#(400, 200)
start_time = time.time()

# feature_names_list = ['Ca1_101073_A_T', 'Ca1_274437_T_C', 'Ca1_293229_G_A', 'Ca1_317478_C_A', 'Ca1_318266_A_G', 'Ca1_318271_G_A', 'Ca1_323797_G_T', 'Ca1_324527_T_G', 'Ca1_350395_T_A', 'Ca1_350494_A_G', 'Ca1_371165_G_T', 'Ca1_371175_T_A', 'Ca1_371216_A_G', 'Ca1_382119_G_C', 'Ca1_393327_C_G', 'Ca1_396182_C_T', 'Ca1_399402_C_A', 'Ca1_414572_G_A', 'Ca1_420648_T_C', 'Ca1_420734_T_C', 'Ca1_422010_T_C', 'Ca1_422611_G_A', 'Ca1_422617_G_A', 'Ca1_422623_T_G', 'Ca1_430734_T_C', 'Ca1_430735_G_A', 'Ca1_431646_A_G', 'Ca1_442737_C_A', 'Ca1_449010_T_C', 'Ca1_461587_T_A', 'Ca1_468077_A_C', 'Ca1_608516_T_G', 'Ca1_608554_C_A', 'Ca1_608649_T_A', 'Ca1_726015_A_C', 'Ca1_737218_T_G', 'Ca1_752961_A_T', 'Ca1_764277_C_T', 'Ca1_771989_A_G', 'Ca1_806263_A_G', 'Ca1_807787_C_T',
# 'Ca1_807822_G_A', 'Ca1_838327_T_C', 'Ca1_845841_C_A', 'Ca1_872675_C_T', 'Ca1_872682_C_A', 'Ca1_882266_C_A', 'Ca1_904772_A_G', 'Ca1_915567_A_T', 'Ca1_927678_C_T', 'Ca1_931602_T_A', 'Ca1_931728_T_C', 'Ca1_934186_T_C', 'Ca1_952261_A_G', 'Ca1_952309_C_T', 'Ca1_966729_C_A', 'Ca1_966776_T_C', 'Ca1_966831_C_T', 'Ca1_970249_T_A', 'Ca1_977274_A_G', 'Ca1_977303_A_G', 'Ca1_977337_C_T', 'Ca1_978810_G_A', 'Ca1_978866_G_A', 'Ca1_998157_A_C', 'Ca1_1001936_G_C', 'Ca1_1001945_T_A', 'Ca1_1023011_C_T', 'Ca1_1023887_A_G', 'Ca1_1024612_A_T', 'Ca1_1032992_C_A', 'Ca1_1032994_A_C', 'Ca1_1033034_A_C', 'Ca1_1055001_A_G', 'Ca1_1063583_G_A', 'Ca1_1067983_T_C', 'Ca1_1077356_A_G', 'Ca1_1080806_G_T', 'Ca1_1099901_T_A', 'Ca1_1113422_C_T', 'Ca1_1113541_A_T', 'Ca1_1118313_G_T', 'Ca1_1123177_T_A', 'Ca1_1123223_C_T', 'Ca1_1128406_T_C', 'Ca1_1128433_T_C', 'Ca1_1128455_A_T', 'Ca1_1128499_A_G', 'Ca1_1155142_C_A', 'Ca1_1167943_G_A', 'Ca1_1197395_A_C', 'Ca1_1197472_C_G', 'Ca1_1214874_T_C', 'Ca1_1224263_T_A', 'Ca1_1224311_T_A', 'Ca1_1224345_G_T', 'Ca1_1224365_C_A', 'Ca1_1242015_T_C', 'Ca1_1248550_C_G', 'Ca1_1270990_T_A', 'Ca1_1276531_C_T', 'Ca1_1284410_G_A', 'Ca1_1288271_A_G', 'Ca1_1295110_A_G', 'Ca1_1337817_G_T', 'Ca1_1338395_A_G', 'Ca1_1338422_G_A',
# 'Ca1_1343655_G_T', 'Ca1_1343729_C_T', 'Ca1_1353790_T_A', 'Ca1_1353798_G_C', 'Ca1_1353876_C_T', 'Ca1_1355093_G_T', 'Ca1_1362732_T_A', 'Ca1_1372278_A_G', 'Ca1_1372301_A_C', 'Ca1_1372317_C_G', 'Ca1_1380295_T_C', 'Ca1_1382930_G_A', 'Ca1_1382969_C_T', 'Ca1_1383024_C_G', 'Ca1_1391129_G_A', 'Ca1_1404820_T_C', 'Ca1_1404848_A_G', 'Ca1_1423164_T_C', 'Ca1_1441722_G_A', 'Ca1_1443612_T_G', 'Ca1_1443614_G_C', 'Ca1_1453072_C_A', 'Ca1_1457542_G_A', 'Ca1_1463497_G_C', 'Ca1_1463579_G_C', 'Ca1_1467331_G_T', 'Ca1_1492296_A_T', 'Ca1_1511501_A_G', 'Ca1_1511523_T_G', 'Ca1_1520328_C_A', 'Ca1_1528052_G_A', 'Ca1_1528162_G_A', 'Ca1_1532918_A_C', 'Ca1_1533118_C_T', 'Ca1_1534227_C_T', 'Ca1_1536432_G_T', 'Ca1_1545721_C_G', 'Ca1_1552406_A_C', 'Ca1_1553267_A_G', 'Ca1_1555825_A_T', 'Ca1_1555831_C_G', 'Ca1_1560796_G_A', 'Ca1_1562218_C_A', 'Ca1_1590875_G_T', 'Ca1_1622307_A_C', 'Ca1_1634236_G_A', 'Ca1_1634261_G_A', 'Ca1_1643959_C_T', 'Ca1_1647611_A_T', 'Ca1_1651662_T_G', 'Ca1_1661336_A_T', 'Ca1_1661337_C_T', 'Ca1_1666655_A_G', 'Ca1_1680426_T_C', 'Ca1_1680454_T_G', 'Ca1_1694313_T_A', 'Ca1_1711149_A_G', 'Ca1_1711166_G_A', 'Ca1_1711285_G_T', 'Ca1_1713338_G_T', 'Ca1_1713341_A_T', 'Ca1_1732351_C_G', 'Ca1_1737275_A_G', 'Ca1_1737311_C_G', 'Ca1_1737321_T_C', 'Ca1_1741298_C_T', 'Ca1_1744087_A_G', 'Ca1_1745468_T_A', 'Ca1_1745498_A_G', 'Ca1_1752930_G_A', 'Ca1_1769208_C_T', 'Ca1_1769313_G_A', 'Ca1_1775071_A_G', 'Ca1_1775117_G_A', 'Ca1_1792406_T_C', 'Ca1_1792412_C_T', 'Ca1_1802612_T_C', 'Ca1_1802655_T_C', 'Ca1_1802686_C_T', 'Ca1_1808643_A_T', 'Ca1_1808654_G_T', 'Ca1_1826905_C_T', 'Ca1_1826915_T_C', 'Ca1_1858664_A_G', 'Ca1_1858717_G_A', 'Ca1_1859502_T_G', 'Ca1_1859532_T_G', 'Ca1_1859541_T_A', 'Ca1_1859659_G_T', 'Ca1_1902178_G_A', 'Ca1_1931844_A_C', 'Ca1_1941178_T_A', 'Ca1_1963329_C_T', 'Ca1_1968309_G_A', 'Ca1_1970785_A_G', 'Ca1_1973210_T_C', 'Ca1_1975152_C_T', 'Ca1_2022225_C_T', 'Ca1_2044937_T_C', 'Ca1_2044940_C_A', 'Ca1_2081846_C_A', 'Ca1_2088933_A_T', 'Ca1_2089358_C_T', 'Ca1_2095150_G_T', 'Ca1_2118239_G_A', 'Ca1_2118249_C_T', 'Ca1_2123393_C_T', 'Ca1_2124581_A_G', 'Ca1_2124830_G_C', 'Ca1_2131479_A_G', 'Ca1_2131625_G_A', 'Ca1_2131662_T_C', 'Ca1_2131671_T_C', 'Ca1_2152476_A_G', 'Ca1_2160865_A_C', 'Ca1_2160933_A_T', 'Ca1_2171127_C_T', 'Ca1_2172623_G_T', 'Ca1_2189525_A_T', 'Ca1_2189645_G_C', 'Ca1_2198562_T_A', 'Ca1_2214749_G_A', 'Ca1_2218420_A_G',
# 'Ca1_2218626_T_G', 'Ca1_2218700_G_A', 'Ca1_2225735_T_A', 'Ca1_2234134_G_A', 'Ca1_2234145_A_G', 'Ca1_2234274_A_C', 'Ca1_2234278_A_T', 'Ca1_2238867_T_A', 'Ca1_2238888_A_C', 'Ca1_2238891_C_T', 'Ca1_2238958_T_C', 'Ca1_2259186_G_A', 'Ca1_2262576_T_C', 'Ca1_2262591_G_A', 'Ca1_2262633_G_T', 'Ca1_2270600_A_G', 'Ca1_2271409_T_C', 'Ca1_2271421_T_C', 'Ca1_2277599_C_T', 'Ca1_2309179_A_G', 'Ca1_2313076_A_T', 'Ca1_2313615_A_G', 'Ca1_2320673_C_A', 'Ca1_2320691_C_T', 'Ca1_2328979_A_G', 'Ca1_2330274_A_C', 'Ca1_2330287_C_G', 'Ca1_2342137_C_A', 'Ca1_2360375_G_T', 'Ca1_2364872_T_C', 'Ca1_2373466_A_G', 'Ca1_2373480_T_G', 'Ca1_2374345_A_C', 'Ca1_2374520_T_A', 'Ca1_2375085_C_T', 'Ca1_2375294_A_T', 'Ca1_2375928_G_A', 'Ca1_2383974_G_C', 'Ca1_2388102_C_T', 'Ca1_2389626_G_A', 'Ca1_2398988_A_G', 'Ca1_2400346_T_C', 'Ca1_2411993_A_G', 'Ca1_2432376_A_G', 'Ca1_2437208_G_T', 'Ca1_2437317_G_A', 'Ca1_2440912_T_A', 'Ca1_2441018_C_T', 'Ca1_2450200_T_C', 'Ca1_2450209_C_T', 'Ca1_2469345_G_A', 'Ca1_2469867_G_A', 'Ca1_2480310_G_A', 'Ca1_2496727_C_T', 'Ca1_2510183_G_C', 'Ca1_2511223_T_A', 'Ca1_2511238_A_G', 'Ca1_2511250_G_A', 'Ca1_2511253_C_T', 'Ca1_2512573_A_T', 'Ca1_2513046_A_G', 'Ca1_2524005_T_C', 'Ca1_2524021_G_C', 'Ca1_2533229_G_A', 'Ca1_2548183_G_T', 'Ca1_2579883_T_C', 'Ca1_2598554_A_G', 'Ca1_2640175_C_A', 'Ca1_2703473_T_G', 'Ca1_2704240_A_G', 'Ca1_2704247_A_G', 'Ca1_2730646_C_G', 'Ca1_2788788_C_G', 'Ca1_2800975_A_C', 'Ca1_2809226_C_T', 'Ca1_2834171_G_A', 'Ca1_2834216_T_A', 'Ca1_2837698_G_T', 'Ca1_2854316_T_A', 'Ca1_2869033_T_A', 'Ca1_2869467_C_T', 'Ca1_2869532_T_A', 'Ca1_2892256_A_G', 'Ca1_2892263_A_C', 'Ca1_2900696_G_A', 'Ca1_2900699_T_C', 'Ca1_2902723_C_T', 'Ca1_2902744_A_C', 'Ca1_2909737_C_A', 'Ca1_2909739_G_A', 'Ca1_2921443_C_A', 'Ca1_2928372_G_A', 'Ca1_2933533_G_A', 'Ca1_2936230_T_C', 'Ca1_2948112_A_C', 'Ca1_2964451_G_A', 'Ca1_2964512_C_A', 'Ca1_2981187_G_A', 'Ca1_2993011_C_A', 'Ca1_2995302_C_T', 'Ca1_3048432_T_C', 'Ca1_3053548_T_C', 'Ca1_3053615_T_C', 'Ca1_3053662_T_C', 'Ca1_3081553_T_A', 'Ca1_3084224_C_T', 'Ca1_3084234_T_A', 'Ca1_3099941_T_C', 'Ca1_3102387_C_T', 'Ca1_3120137_A_G', 'Ca1_3120143_G_A', 'Ca1_3120163_A_G', 'Ca1_3120165_C_A', 'Ca1_3142432_C_T', 'Ca1_3302504_C_G', 'Ca1_3953643_A_G', 'Ca1_4235091_C_A', 'Ca1_4244360_A_G', 'Ca1_4249113_C_T', 'Ca1_4379745_C_A', 'Ca1_4474445_C_G', 'Ca1_4518653_A_T', 'Ca1_4766059_A_T',
# 'Ca1_4782895_C_T', 'Ca1_5124322_C_A', 'Ca1_5367087_A_C', 'Ca1_5371236_C_T', 'Ca1_5391354_T_G', 'Ca1_5391371_C_T', 'Ca1_5428038_T_C', 'Ca1_5428080_T_A', 'Ca1_5441052_T_C', 'Ca1_5441097_C_T', 'Ca1_5480709_G_C', 'Ca1_5511919_C_T', 'Ca1_5511968_A_C', 'Ca1_5511996_C_A', 'Ca1_5515026_T_C', 'Ca1_5614996_A_G', 'Ca1_5619968_A_T', 'Ca1_5632070_G_T', 'Ca1_5638251_G_A', 'Ca1_5638273_T_C', 'Ca1_5645640_A_G', 'Ca1_5720006_T_A', 'Ca1_5720035_T_A', 'Ca1_5731384_T_C', 'Ca1_5742731_G_A', 'Ca1_5745277_G_A', 'Ca1_5757141_C_T', 'Ca1_5758008_T_A', 'Ca1_5963423_C_T', 'Ca1_6132150_C_T', 'Ca1_6138577_T_C', 'Ca1_6138657_A_G', 'Ca1_6140949_G_A', 'Ca1_6144083_A_G', 'Ca1_6151476_G_C', 'Ca1_6154711_G_A', 'Ca1_6155943_G_C', 'Ca1_6155945_T_G', 'Ca1_6162693_T_C', 'Ca1_6165883_C_A', 'Ca1_6277507_T_C', 'Ca1_6277549_A_G', 'Ca1_6282159_C_A', 'Ca1_6314770_A_C', 'Ca1_6327680_C_A', 'Ca1_6327709_G_T', 'Ca1_6330555_A_G']

nn = math.ceil(cp.sqrt(len(feature_names_list)))      			     # Image dimension

Nn = len(feature_names_list)                                                             # Number of features
Euc_Dist = cp.asarray(euclidean_distances(X))
Euc_Dist = cp.maximum(Euc_Dist, Euc_Dist.transpose())





In [ ]:
# Euc_Dist = cp.asarray(Euc_Dist)
mds_xy = myMDS(Euc_Dist)

eq_xy = two_d_eq(mds_xy, Nn)                                        # -> [0,1]
Img = Assign_features_to_pixels(eq_xy,nn,verbose=1)
print(Img)
				# Img is the none-overlapping coordinates generated by MDS
# Dist = pd.DataFrame(data = Euc_Dist, columns = feature_names_list, index = feature_names_list)	# Generating a distance matrix which includes the Euclidean distance between each and every descriptor

# np.save("/content/drive/MyDrive/Диплом/data/eq_xy.npy", eq_xy)
# np.save("/content/drive/MyDrive/Диплом/data/Img.npy", Img)

<class 'cupy.ndarray'>


In [21]:

eq_xy = np.load("/content/drive/MyDrive/Диплом/data/eq_xy.npy")
Img = np.load("/content/drive/MyDrive/Диплом/data/Img.npy", allow_pickle=True)

In [29]:
# Convert from 'F34' to int 34
# init_map = np.char.strip(Img.astype(str),'F').astype(int)
map_in_int = Img


for iter_num in range(iters_count):
    init_coords = [x for x in product([0,1,2],repeat = 2)]
    for init_coord in init_coords:
        # generate the centroids
        xxx = [init_coord[0]+i*3 for i in range(int(nn/3)+1) if (init_coord[0]+i*3)<nn]
        yyy = [init_coord[1]+i*3 for i in range(int(nn/3)+1) if (init_coord[1]+i*3)<nn]
        centr_list = [x for x in product(xxx,yyy)]
        # print(map_in_int)
        swap_dict = evaluate_centroids_in_list(centr_list,Euc_Dist,map_in_int)
        # print(swap_dict)
        print(swap_dict)
        map_in_int = execute_dict_swap(swap_dict, map_in_int)

        print(">",init_coord,"Corr:",universial_corr(Euc_Dist,map_in_int))
        print()


coords = np.array([[item[0] for item in np.where(map_in_int == ii)] for ii in range(Nn)])

X_REFINED_MDS = REFINED_Im_Gen(X[:,:],nn, map_in_int, feature_names_list, coords)


X_reshaped = X_REFINED_MDS.reshape(X_REFINED_MDS.shape[0], nn, nn)
if not os.path.exists(path):
    os.mkdir(path)
for i in range(X_reshaped.shape[0]):
    plt.imsave(f"""{path}/img{i}.png""", X_reshaped[i])
print(X_reshaped.shape)
end_time = time.time()
execution_time = end_time - start_time

print("Время выполнения кода: ", execution_time, "секунд")





[[5. 1. 7.]
 [8. 3. 6.]
 [4. 2. 0.]]
0
(array([2]), array([2]))
1
(array([0]), array([1]))
2
(array([2]), array([1]))
3
(array([1]), array([1]))
4
(array([2]), array([0]))
5
(array([0]), array([0]))
6
(array([1]), array([2]))
7
(array([0]), array([2]))
8
(array([1]), array([0]))

[[5. 1. 7.]
 [8. 3. 6.]
 [4. 2. 0.]]
0
(array([2]), array([2]))
1
(array([0]), array([1]))
2
(array([2]), array([1]))
3
(array([1]), array([1]))
4
(array([2]), array([0]))
5
(array([0]), array([0]))
6
(array([1]), array([2]))
7
(array([0]), array([2]))
8
(array([1]), array([0]))

[[1. 1. 7.]
 [8. 3. 6.]
 [4. 2. 0.]]
0
(array([2]), array([2]))
1
(array([0, 0]), array([0, 1]))
2
(array([2]), array([1]))
3
(array([1]), array([1]))
4
(array([2]), array([0]))
5
(array([], dtype=int64), array([], dtype=int64))
6
(array([1]), array([2]))
7
(array([0]), array([2]))
8
(array([1]), array([0]))



TypeError: Implicit conversion to a NumPy array is not allowed. Please use `.get()` to construct a NumPy array explicitly.

In [ ]:
np.save('allimg.npy', X_reshaped)

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968